In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle
import spacy

# Language modeling

## Grabbing data for the model

In [7]:
PATH = 'data/aclImdb/'

TRAIN_PATH = 'train/all/'
VALIDATION_PATH = 'test/all/'
TRAIN = f'{PATH}{TRAIN_PATH}'
VALIDATION = f'{PATH}{TRAIN_PATH}'

% ls {PATH}

imdbEr.txt  imdb.vocab  README  test/  train/


In [8]:
training_files = ! ls {TRAIN}
training_files[:10]

['0_0.txt',
 '0_3.txt',
 '0_9.txt',
 '10000_0.txt',
 '10000_4.txt',
 '10000_8.txt',
 '1000_0.txt',
 '10001_0.txt',
 '10001_10.txt',
 '10001_4.txt']

In [11]:
review = ! cat {TRAIN}{training_files[6]}
review[0]

"I have to say when a name like Zombiegeddon and an atom bomb on the front cover I was expecting a flat out chop-socky fung-ku, but what I got instead was a comedy. So, it wasn't quite was I was expecting, but I really liked it anyway! The best scene ever was the main cop dude pulling those kids over and pulling a Bad Lieutenant on them!! I was laughing my ass off. I mean, the cops were just so bad! And when I say bad, I mean The Shield Vic Macky bad. But unlike that show I was laughing when they shot people and smoked dope.<br /><br />Felissa Rose...man, oh man. What can you say about that hottie. She was great and put those other actresses to shame. She should work more often!!!!! I also really liked the fight scene outside of the building. That was done really well. Lots of fighting and people getting their heads banged up. FUN! Last, but not least Joe Estevez and William Smith were great as the...well, I wasn't sure what they were, but they seemed to be having fun and throwing out 

In [33]:
# not sure why this is giving me the wrong answer, or at least different from command-line version
# ! find {TRAIN} -name '*.txt' | xargs wc -w | tail

In [24]:
! find {TRAIN} -name '*.txt' | xargs cat | wc -w

17486581


In [34]:
spacy_tokens = spacy.load('en')

OSError: [E050] Can't find model 'en'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.